In [3]:
import sys
sys.path.append('/projects/sign_language/')

In [4]:
import numpy as np

import pytorchvideo.data

import pathlib
from transformers import VideoMAEFeatureExtractor


2023-01-11 10:56:28.479289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 10:56:31.188570: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-11 10:56:31.188914: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist

In [5]:

model_ckpt = "MCG-NJU/videomae-base"
feature_extractor = VideoMAEFeatureExtractor.from_pretrained(model_ckpt)



/usr/local/lib/python3.8/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [6]:
folds = 3


In [12]:
for fold in range(folds):
    dataset_root_path = '/projects/ZHO/formats/letters_splitted/'+str(fold)
    dataset_root_path = pathlib.Path(dataset_root_path)    
    video_count_train = len(list(dataset_root_path.glob("train/*/*.mp4")))
    video_count_val = len(list(dataset_root_path.glob("val/*/*.mp4")))
    video_count_test = len(list(dataset_root_path.glob("test/*/*.mp4")))
    video_total = video_count_train + video_count_val + video_count_test
    print(video_count_train, video_count_val, video_count_test)
    print(f"Total videos: {video_total}")

64 32 32
Total videos: 128
64 32 32
Total videos: 128
64 32 32
Total videos: 128


In [13]:

all_video_file_paths = (
    list(dataset_root_path.glob("train/*/*.mp4"))
    + list(dataset_root_path.glob("val/*/*.mp4"))
    + list(dataset_root_path.glob("test/*/*.mp4"))
)


In [20]:

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)




In [21]:


import os

mean = feature_extractor.image_mean
std = feature_extractor.image_std
resize_to = feature_extractor.size['shortest_edge']
print(resize_to)
num_frames_to_sample = 16#model.config.num_frames
sample_rate = 4
fps = 30
#clip_duration = num_frames_to_sample * sample_rate / fps
clip_duration =10
print(clip_duration)




224
10


In [22]:

# Training dataset transformations.
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    #RandomShortSideScale(min_size=256, max_size=320),
                    Resize((resize_to,resize_to)),
                    #RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)


# Validation and evaluation datasets' transformations.
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize((resize_to, resize_to)),
                ]
            ),
        ),
    ]

)

# Training dataset.


In [23]:

from torch.utils.data import Dataset
import torch

from torch.utils.data import DataLoader


In [24]:
folds = ['0','1','2']
for fold in folds:
    dataset_root_path = '/projects/ZHO/formats/letters_splitted/'+fold
    dataset_root_path = pathlib.Path(dataset_root_path)
    train_dataset = pytorchvideo.data.labeled_video_dataset(
        data_path=os.path.join(dataset_root_path, "train"),
        #data_path = os.path.join(dataset_root_path,'annotations','trainlist01.txt'),
        clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
        decode_audio=False,
        transform=train_transform,
    )

    test_dataset = pytorchvideo.data.labeled_video_dataset(
        data_path=os.path.join(dataset_root_path, "test"),
        #data_path = os.path.join(dataset_root_path,'annotations','trainlist01.txt'),
        clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
        decode_audio=False,
        transform=val_transform,
    )


    train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle = False, num_workers=32)  
    test_dataloader = DataLoader(test_dataset,batch_size=32,num_workers=32)  

    big_tensor_video_train = np.zeros((64,3,16,224,224))
    #big_tensor_label_train = {}
    big_tensor_video_train_list = {k:[] for k in range(32)}

    big_tensor_video_test = np.zeros((32,3,16,224,224))
    #big_tensor_label_test = []


    for i,j in enumerate(train_dataloader):

    #    input = j['video'].permute(0,2,1,3,4)

        input = j['video'].numpy()
        labels = j['label'].numpy()

        for k in range(input.shape[0]):

            big_tensor_video_train_list[labels[k]].append(input[k])
        
        #big_tensor_video_train.append(input)    
        print(labels)

    for k in range(len(big_tensor_video_train_list)):
        for jj in range(2):
            big_tensor_video_train[2*k+jj]=big_tensor_video_train_list[k][jj]

    for i,j in enumerate(test_dataloader):
    #    input = j['video'].permute(0,2,1,3,4)

        input = j['video'].numpy()
        labels = j['label'].numpy()

        for k in range(input.shape[0]):

            big_tensor_video_test[labels[k]] = input[k]
        print(labels)

    big_tensor_video_train_reshaped = np.transpose(big_tensor_video_train, (0,2,1,3,4))
    big_tensor_video_test_reshaped  = np.transpose(big_tensor_video_test, (0,2,1,3,4))

    np.save('train_video_letters_only_data_'+fold+'.npy',big_tensor_video_train_reshaped)
    np.save('test_video_letters_only_data'+fold+'.npy',big_tensor_video_test_reshaped)




[12 11]
[ 4 26]
[28 25]
[19 16]
[ 9 24]
[ 7 25]
[29 23]
[28 26]
[21  3]
[23  2]
[30 27]
[20 21]
[15 22]
[18 17]
[27  8]
[7 1]
[31  5]
[11  2]
[16 13]
[24  4]
[6 8]
[19 17]
[ 0 30]
[14  9]
[ 0 18]
[15  5]
[1 6]
[ 3 10]
[14 22]
[31 29]
[20 10]
[13 12]
[7]
[19]
[26]
[5]
[17]
[24]
[23]
[9]
[12]
[16]
[18]
[10]
[4]
[0]
[3]
[25]
[28]
[20]
[21]
[8]
[6]
[1]
[11]
[15]
[14]
[27]
[30]
[29]
[31]
[2]
[22]
[13]
[22  9]
[21 12]
[12  1]
[23 14]
[19  2]
[25 21]
[ 2 29]
[18 14]
[31  5]
[25 26]
[30  5]
[20 13]
[ 8 19]
[29  4]
[23 16]
[20  0]
[10 13]
[16  6]
[18  3]
[ 7 27]
[28 28]
[ 4 15]
[ 0 17]
[11 10]
[30  8]
[27 24]
[11 26]
[1 3]
[ 6 22]
[ 7 17]
[15 24]
[ 9 31]
[26]
[13]
[0]
[22]
[7]
[6]
[29]
[10]
[19]
[28]
[30]
[20]
[18]
[2]
[4]
[5]
[1]
[23]
[3]
[11]
[9]
[8]
[14]
[17]
[25]
[31]
[16]
[15]
[21]
[12]
[24]
[27]
[31  4]
[17  9]
[26 16]
[25 18]
[2 8]
[13 28]
[29  9]
[5 6]
[18  1]
[15 15]
[ 8 13]
[ 6 28]
[ 0 25]
[31  4]
[ 7 11]
[ 2 21]
[20 30]
[ 5 20]
[24 19]
[22 14]
[12 27]
[ 3 29]
[19  7]
[ 0 23]
[21 14]
